# Additional End of week Exercise - week 2

Now use everything you've learned from Week 2 to build a full prototype for the technical question/answerer you built in Week 1 Exercise.

This should include a Gradio UI, streaming, use of the system prompt to add expertise, and the ability to switch between models. Bonus points if you can demonstrate use of a tool!

If you feel bold, see if you can add audio input so you can talk to it, and have it respond with audio. ChatGPT or Claude can help you, or email me if you have questions.

I will publish a full solution here soon - unless someone beats me to it...

There are so many commercial applications for this, from a language tutor, to a company onboarding solution, to a companion AI to a course (like this one!) I can't wait to see your results.

In [ ]:
#Imports
from IPython.display import Markdown, display
from openai import OpenAI
import os
import json
import requests
import gradio as gr
from dotenv import load_dotenv
from typing import List
import time
from datetime import datetime, timedelta
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import json
import re


In [ ]:
OLLAMA_BASE_URL="http://localhost:11434/v1/completions"
LOCAL_MODEL_NAME="llama3.2"


# Load environment variables in a file called .env

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')
OPENAI_API_KEY=api_key

load_dotenv(override=True)
coin_key = os.getenv('COINMARKETCAP_API_KEY')
COINMARKETCAP_API_KEY = coin_key

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

In [ ]:
# Ollama configuration
OLLAMA_URL = os.getenv("OLLAMA_BASE_URL", "http://localhost:11434/v1/completions")
OLLAMA_MODEL = os.getenv("LOCAL_MODEL_NAME", "llama3.2")

# OpenAI configuration
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_MODEL = "gpt-4"

In [ ]:
# Crypto Analysis Prompt
CRYPTO_SYSTEM_PROMPT = """You are a specialized AI assistant with expertise in cryptocurrency markets and data analysis.
Your role is to help users identify and understand cryptocurrencies with the strongest growth patterns over recent weeks.
Provide clear, data-driven insights about market trends and performance metrics."""


In [ ]:

def scrape_coingecko(limit=10, debug=False):
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
            'Accept-Language': 'en-US,en;q=0.5',
            'Referer': 'https://www.coingecko.com/'
        }

        url = "https://www.coingecko.com/en/coins/trending"
        response = requests.get(url, headers=headers, timeout=30)
        response.raise_for_status()

        if debug:
            print(f"Status: {response.status_code}")
            with open("debug_coingecko.html", "w", encoding="utf-8") as f:
                f.write(response.text)
            print("HTML saved to debug_coingecko.html")

        soup = BeautifulSoup(response.content, 'html.parser')
        top_performers = []

        # Try multiple selectors
        rows = (soup.find_all('tr', {'data-sort-by': True}) or
                soup.find_all('tr', class_=re.compile('hover')) or
                soup.select('table tbody tr'))[:limit]

        if debug:
            print(f"Found {len(rows)} rows")

        for row in rows:
            try:
                # Find all text in row
                texts = [t.strip() for t in row.stripped_strings]
                if debug:
                    print(f"Row texts: {texts[:5]}")

                # Extract data from text list
                name = texts[1] if len(texts) > 1 else "Unknown"
                symbol = texts[2] if len(texts) > 2 else "N/A"

                # Find price
                price = 0
                for text in texts:
                    if '$' in text:
                        price_str = text.replace('$', '').replace(',', '')
                        try:
                            price = float(price_str)
                            break
                        except:
                            continue

                # Find percentage change
                change_30d = 0
                for text in texts:
                    if '%' in text:
                        change_str = text.replace('%', '').replace('+', '')
                        try:
                            change_30d = float(change_str)
                        except:
                            continue

                if name != "Unknown":
                    top_performers.append({
                        "name": name,
                        "symbol": symbol,
                        "current_price": price,
                        "price_change_percentage_30d": change_30d,
                        "source": "coingecko"
                    })
            except Exception as e:
                if debug:
                    print(f"Row error: {e}")
                continue

        return {"timeframe": "30d", "timestamp": datetime.now().isoformat(), "count": len(top_performers), "top_performers": top_performers}
    except Exception as e:
        return {"error": str(e)}



def get_top_performers(source="coingecko", limit=10, save=False, debug=False):
    sources = {"coingecko": scrape_coingecko, "coinmarketcap": scrape_coinmarketcap}
    result = sources[source](limit, debug)

    if save and "error" not in result:
        filename = f"crypto_{source}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
        with open(filename, 'w') as f:
            json.dump(result, f, indent=2)
        print(f"Saved to {filename}")

    return result

if __name__ == "__main__":
    print("Testing CoinGecko with debug...")
    result = get_top_performers("coingecko", 10, True, debug=True)
    print(json.dumps(result, indent=2))

    print("\n" + "="*60 + "\n")

    print("Testing CoinMarketCap with debug...")
    result = get_top_performers("coinmarketcap", 10, True, debug=True)
    print(json.dumps(result, indent=2))

In [ ]:
def scrape_coinmarketcap(limit=10, debug=False):
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
            'Accept-Language': 'en-US,en;q=0.5',
        }

        url = "https://coinmarketcap.com/gainers-losers/"
        response = requests.get(url, headers=headers, timeout=30)
        response.raise_for_status()

        if debug:
            print(f"Status: {response.status_code}")
            with open("debug_coinmarketcap.html", "w", encoding="utf-8") as f:
                f.write(response.text)
            print("HTML saved to debug_coinmarketcap.html")

        soup = BeautifulSoup(response.content, 'html.parser')
        top_performers = []

        # Find all table rows
        rows = soup.find_all('tr')
        if debug:
            print(f"Total rows found: {len(rows)}")

        for row in rows[1:limit+1]:
            try:
                texts = [t.strip() for t in row.stripped_strings]
                if debug and len(texts) > 0:
                    print(f"Row texts: {texts[:5]}")

                if len(texts) < 3:
                    continue

                # Usually: rank, name, symbol, price, change...
                name = texts[1] if len(texts) > 1 else "Unknown"
                symbol = texts[2] if len(texts) > 2 else "N/A"

                price = 0
                change_30d = 0

                for text in texts:
                    if '$' in text and price == 0:
                        try:
                            price = float(text.replace('$', '').replace(',', ''))
                        except:
                            continue
                    if '%' in text:
                        try:
                            change_30d = float(text.replace('%', '').replace('+', ''))
                        except:
                            continue

                if name != "Unknown":
                    top_performers.append({
                        "name": name,
                        "symbol": symbol,
                        "current_price": price,
                        "price_change_percentage_30d": change_30d,
                        "source": "coinmarketcap"
                    })
            except Exception as e:
                if debug:
                    print(f"Row error: {e}")
                continue

        return {"timeframe": "30d", "timestamp": datetime.now().isoformat(), "count": len(top_performers), "top_performers": top_performers}
    except Exception as e:
        return {"error": str(e)}

In [ ]:


# Tool detection and execution
def detect_and_run_tool(user_message: str):
    user_message_lower = user_message.lower().strip()

    # Detect crypto growth queries
    crypto_keywords = ["crypto growth", "top gainers", "best performing", "crypto performance", "trending coins"]

    if any(keyword in user_message_lower for keyword in crypto_keywords):
        return True, get_top_performers("coingecko", 10, True, debug=True)




In [ ]:
def ask_ollama(prompt: str) -> str:
    try:
        payload = {"model": OLLAMA_MODEL, "prompt": prompt, "stream": False}
        r = requests.post(OLLAMA_URL, json=payload, timeout=120)
        r.raise_for_status()
        data = r.json()
        return data.get("choices", [{}])[0].get("text", "").strip()
    except Exception as e:
        return f"[Ollama error: {e}]"

In [ ]:
def ask_openai(prompt: str) -> str:
    try:
        from openai import OpenAI
        client = OpenAI(api_key=OPENAI_API_KEY)

        response = client.chat.completions.create(
            model=OPENAI_MODEL,
            messages=[
                {"role": "system", "content": CRYPTO_SYSTEM_PROMPT},
                {"role": "user", "content": prompt}
            ],
            max_tokens=512,
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"[OpenAI error: {e}]"

In [ ]:
def chat_fn(user_message: str, history: List[List[str]], model_choice: str):
    tool_used, tool_output = detect_and_run_tool(user_message)

    if tool_used:
        if "error" in tool_output:
            reply = f"Data fetch error: {tool_output['error']}"
        else:
            # Format the crypto data for AI analysis
            crypto_data_str = json.dumps(tool_output, indent=2)

            # Create analysis prompt
            analysis_prompt = f"""
            Analyze this cryptocurrency growth data and provide insights:

            {crypto_data_str}

            Please identify:
            1. The strongest performers and their growth patterns
            2. Any notable trends across different timeframes
            3. Risk considerations or notable observations
            4. Simple, actionable insights for the user

            Keep the analysis clear and data-driven.
            User's original question: {user_message}
            """

            # Get AI analysis
            if model_choice == "openai":
                analysis = ask_openai(analysis_prompt)
            else:
                ollama_prompt = f"{CRYPTO_SYSTEM_PROMPT}\n\nUser: {analysis_prompt}\nAssistant:"
                analysis = ask_ollama(ollama_prompt)

            reply = f"📊 **Crypto Growth Analysis**\n\n{analysis}\n\n*Raw data for reference:*\n```json\n{crypto_data_str}\n```"

    else:
        # Regular conversation
        if model_choice == "openai":
            reply = ask_openai(user_message)
        else:
            prompt = f"{CRYPTO_SYSTEM_PROMPT}\n\nUser: {user_message}\nAssistant:"
            reply = ask_ollama(prompt)

    history.append([user_message, reply])
    return history

# Enhanced Gradio UI with crypto focus
def main():
    with gr.Blocks(title="Crypto Growth Analyst Chatbot") as demo:
        gr.Markdown("""
        # Samuel Week 2 Task: Crypto Growth Analyst Chatbot
        **Analyze cryptocurrency performance with dual AI models** (Ollama & OpenAI)

        *Try questions like:*
        - "Show me cryptocurrencies with strongest growth"
        - "What are the top performing coins this month?"
        - "Analyze crypto market trends"
        """)

        # Message input
        msg = gr.Textbox(
            placeholder="Ask about crypto growth trends or type /ticket <city>",
            label="Your message",
            lines=2,
            autofocus=True
        )

        # Model selection
        with gr.Row():
            model_choice = gr.Radio(
                ["ollama", "openai"],
                value="ollama",
                label="AI Model"
            )
            send = gr.Button("Analyze Crypto Data", variant="primary")

        # Chatbot area
        chatbot = gr.Chatbot(label="Crypto Analysis Conversation", height=500, type="messages")

        # Wrapper function
        def wrapped_chat_fn(user_message, history, model_choice):
            updated_history = chat_fn(user_message, history, model_choice)
            return updated_history, gr.update(value="")

        # Event handlers
        send.click(wrapped_chat_fn, inputs=[msg, chatbot, model_choice], outputs=[chatbot, msg])
        msg.submit(wrapped_chat_fn, inputs=[msg, chatbot, model_choice], outputs=[chatbot, msg])

    demo.launch(server_name="0.0.0.0", share=False)

if __name__ == "__main__":
    main()

    